# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

# Loading Data

In [11]:
data = pd.read_excel("bank_statement.xlsx")
print(data.dtypes) # Display data types
print(data.head(3))

ID                                int64
Time                     datetime64[ns]
Amount of money (VND)             int64
Vendor                           object
Latitude                        float64
Longitude                       float64
IP Address                       object
Device’s Address                 object
dtype: object
   ID                Time  Amount of money (VND)     Vendor   Latitude  \
0   1 2020-09-02 18:43:06                1000000       Tiki  10.772109   
1   2 2020-09-05 20:40:00                2000000      Apple  20.837134   
2   3 2020-09-06 05:26:03                3000000  Microsoft  38.897710   

    Longitude     IP Address   Device’s Address  
0  106.698278  118.69.35.205  F4:5C:89:A3:03:A7  
1  106.686365   156.33.241.5  A8:20:66:58:8D:41  
2  -77.036487     13.32.32.0  BC:3D:85:22:19:BC  


# Cleaning Data

In [12]:
def clean_data(df):
    # Create a copy of the data
    data_clean = df.copy()
    # Remove missing values
    data_clean = df.dropna()
    # Remove duplicates
    data_clean = df.drop_duplicates(subset="ID", keep= "first")
    return data_clean
data_clean = clean_data(data)
print(data_clean.head(3))

   ID                Time  Amount of money (VND)     Vendor   Latitude  \
0   1 2020-09-02 18:43:06                1000000       Tiki  10.772109   
1   2 2020-09-05 20:40:00                2000000      Apple  20.837134   
2   3 2020-09-06 05:26:03                3000000  Microsoft  38.897710   

    Longitude     IP Address   Device’s Address  
0  106.698278  118.69.35.205  F4:5C:89:A3:03:A7  
1  106.686365   156.33.241.5  A8:20:66:58:8D:41  
2  -77.036487     13.32.32.0  BC:3D:85:22:19:BC  


# Transforming coordinates to location

In [ ]:
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent="geoapi")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1, error_wait_seconds=2, max_retries=2, swallow_exceptions=True)

# Create a location column
data_clean["Location"] = np.nan

# get the location base on coordinate
for i in range(len(data_clean)):
    lat = data_clean.loc[i, "Latitude"]
    lon = data_clean.loc[i, "Longitude"]

    # check valid coordinate
    valid = (
        pd.notna(lat) and pd.notna(lon) and
        (-90.0 <= float(lat) <= 90.0) and
        (-180.0 <= float(lon) <= 180.0)
    )

    if not valid:
        data_clean.loc[i, "Location"] = "Invalid"
        continue

    # geopy.reverse
    loc = reverse((float(lat), float(lon)), exactly_one=True, language="en", timeout=10) # only get the exactest location
    data_clean.loc[i, "Location"] = loc.address if loc else "Not found" # if not found, return "Not found"
print(data_clean)

C:\Users\Khanh Hoa\AppData\Local\Temp\ipykernel_5544\577935205.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Ben Thanh Market, Quach Thi Trang Square, Khu phố 8, Phường Bến Thành, Ho Chi Minh City, 71009, Vietnam' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data_clean.loc[i, "Location"] = loc.address if loc else "Not found" # if not found, return "Not found"


   ID                Time  Amount of money (VND)     Vendor   Latitude  \
0   1 2020-09-02 18:43:06                1000000       Tiki  10.772109   
1   2 2020-09-05 20:40:00                2000000      Apple  20.837134   
2   3 2020-09-06 05:26:03                3000000  Microsoft  38.897710   
3   4 2020-09-10 08:06:50                3000000       Nike  48.794301   
4   5 2020-09-15 22:40:00                3000000     Amazon  38.897575   
5   6 2020-09-16 21:15:00                3200000     Adidas -30.418506   
6   7 2020-09-18 23:40:00                3500000     Chanel -92.253425   
7   8 2020-09-19 20:42:00                2000000     Lazada  38.897197   
8   9 2020-09-20 09:58:00                5000000       Puma  10.772836   
9  10 2020-09-25 03:56:00                2500000     Google -37.809677   

    Longitude       IP Address                      Device’s Address  \
0  106.698278    118.69.35.205                     F4:5C:89:A3:03:A7   
1  106.686365     156.33.241.5           

# Transforming IP address to location

In [ ]:
import requests

# create a new column
data_clean["IP Geolocation"] = np.nan
# get the location base on coordinate
for i in range(len(data_clean)):
    ip = data_clean.loc[i, "IP Address"]
    url = f"http://ip-api.com/json/{ip}"
    
    try:
        response = requests.get(url, timeout=5)
        ip_loc = response.json()
        # if the status is success, get the city and country
        if ip_loc.get("status") == "success":
            city = ip_loc.get("city", "Unknown")
            country = ip_loc.get("country", "Unknown")
            data_clean.loc[i, "IP Geolocation"] = f"{city}, {country}"
        else: 
            data_clean.loc[i, "IP Geolocation"] = "Not found"
    except Exception as e: # if there is an error, return the error message
        data_clean.loc[i, "IP Geolocation"] = f"Error: {e}"
# export to excel file
data_clean.to_excel("data_clean.xlsx", index=False)

C:\Users\Khanh Hoa\AppData\Local\Temp\ipykernel_5544\2311241570.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Ho Chi Minh City, Vietnam' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data_clean.loc[i, "IP Geolocation"] = f"{city}, {country}"
